In [17]:
!pip install pytorch-pretrained-bert==0.6.2
!pip install git+https://github.com/boudinfl/pke.git
!pip install flashtext
!python -m spacy download en
!python -m nltk.downloader universal_tagset

  Cloning https://github.com/boudinfl/pke.git to /private/var/folders/yc/lv5d2jcd0j12d6nbg3p37hzw0000gn/T/pip-req-build-00vfkq1t
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /private/var/folders/yc/lv5d2jcd0j12d6nbg3p37hzw0000gn/T/pip-req-build-00vfkq1t
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package universal_tagset to

In [21]:
import re
import torch
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM

# Loading pre-trained model
import time
start = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
end = time.time()
print ("Time Elapsed to load BERT ",end-start)

Time Elapsed to load BERT  8.52057671546936


In [24]:
def get_predicted_words(text):
    text = "[CLS] " + text.replace("____", "[MASK]") + " [SEP]"
    # classification and seperation use as a mark of beginning and ending. input format used in bert
    tokenized_text = tokenizer.tokenize(text)
    #print("tokenized sentence: ",tokenized_text,"\n")
    masked_index = tokenized_text.index('[MASK]')
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    # Get 40 choices for the masked(blank) word 
    k = 40
    predicted_index, predicted_index_values = torch.topk(predictions[0, masked_index], k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_index_values.tolist())
    filtered_tokens_to_remove_punctuation = []
    # Removing predictions that contain punctuation etc as they are not relevant.
    for token in predicted_tokens:
        if re.match("^[a-zA-Z0-9_]*$", token):
            filtered_tokens_to_remove_punctuation.append(token)
        
    return filtered_tokens_to_remove_punctuation

In [25]:
sentence = "India is a ____ country."
print ("original sentence: ",sentence,"\n")
predicted_words = get_predicted_words(sentence)
print ("predicted choices: ", predicted_words)

original sentence:  India is a ____ country. 

predicted choices:  ['developing', 'muslim', 'communist', 'democratic', 'developed', 'backward', 'sovereign', 'free', 'socialist', 'small', 'foreign', 'multicultural', 'colonial', 'new', 'christian', 'commonwealth', 'buddhist', 'poor', 'neutral', 'different', 'friendly', 'federal', 'capitalist', 'distinct', 'hindu', 'union', 'diverse', 'western', 'rich', 'growing', 'minority', 'migratory', 'separate', 'dependent', 'backwards', 'multinational', 'cooperative', 'beautiful', 'catholic', 'border']


In [1]:
file_path ="/Users/jasmithamareddy/Desktop/example.txt" 

def read_file(file_path):
    with open(file_path, 'r') as content_file:
        content = content_file.read()
        return content
    
text = read_file(file_path)
print(text)

The short story is a crafted form in its own right. Short stories make use of plot, resonance and other dynamic components as in a novel, but typically to a lesser degree. While the short story is largely distinct from the novel or novella/short novel, authors generally draw from a common pool of literary techniques.[citation needed] The short story is sometimes referred to as a genre Determining what exactly defines a short story remains problematic. A classic definition of a short story is that one should be able to read it in one sitting, a point most notably made in Edgar Allan Poe's essay "The Philosophy of Composition" (1846) H.G. Wells described the purpose of the short story as "The jolly art, of making something very bright and moving; it may be horrible or pathetic or funny or profoundly illuminating, having only this essential, that it should take from fifteen to fifty minutes to read aloud." According to William Faulkner, a short story is character-driven and a writer's job

In [2]:
import pke
import string
import nltk
from nltk.tokenize import word_tokenize

# Downloading NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def get_keyphrases(file_path):
    out = []

    extractor = pke.unsupervised.MultipartiteRank()
    
    # Reading text from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    extractor.load_document(input=text)
    
    # Tokenizing the text and perform pos tagging
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    
    # Filter tokens tagged as adpositions
    adpositions = [token[0] for token in pos_tags if token[1] == 'IN']
    
    # Candidate selection without specifying stoplist
    extractor.candidate_selection()
    
    # Build the Multipartite graph and rank candidates using random walk,
    # alpha controls the weight adjustment mechanism
    extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')
    
    # Get the top-ranked keyphrases
    keyphrases = extractor.get_n_best(n=10)
    
    # Filter out adpositions from keyphrases
    for key in keyphrases:
        if key[0] not in adpositions:
            out.append(key[0])
    
    return out


file_path = "/Users/jasmithamareddy/Desktop/example.txt"

keyphrases = get_keyphrases(file_path)
print("Keyphrases from the text:", keyphrases)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jasmithamareddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jasmithamareddy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Keyphrases from the text: ['short story', 'novel', 'point', 'form', 'story', 'classic definition', 'plot', 'jolly art', 'climax', 'genre']


In [3]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_sentences = {keyword: [] for keyword in keywords}
    for sentence in sentences:
        for keyword in keywords:
            if keyword in sentence:
                keyword_sentences[keyword].append(sentence)
    return keyword_sentences

sentences = tokenize_sentences(text)

keyword_sentence_mapping = get_sentences_for_keyword(keyphrases, sentences)

for keyword, sentences in keyword_sentence_mapping.items():
    print("Keyword:", keyword)
    print("Sentences:")
    for sentence in sentences:
        print("-", sentence)
    print()



Keyword: short story
Sentences:
- The short story is a crafted form in its own right.
- While the short story is largely distinct from the novel or novella/short novel, authors generally draw from a common pool of literary techniques.
- [citation needed] The short story is sometimes referred to as a genre Determining what exactly defines a short story remains problematic.
- A classic definition of a short story is that one should be able to read it in one sitting, a point most notably made in Edgar Allan Poe's essay "The Philosophy of Composition" (1846) H.G.
- Wells described the purpose of the short story as "The jolly art, of making something very bright and moving; it may be horrible or pathetic or funny or profoundly illuminating, having only this essential, that it should take from fifteen to fifty minutes to read aloud."
- According to William Faulkner, a short story is character-driven and a writer's job is to "...trot along behind him with a paper and pencil trying to keep up 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jasmithamareddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import random

def generate_mcqs_with_answers(keyword_sentence_mapping, keyphrases):
    mcqs_with_answers = []
    for keyword, sentences in keyword_sentence_mapping.items():
        if len(sentences) > 0:
            for sentence in sentences:
                # Replace the keyword with a blank
                question = sentence.replace(keyword, "____")
                
                # Determine the number of options randomly, between 2 and 4
                num_options = random.randint(2, min(4, len(keyphrases)))
                
                # Shuffle the keyphrases and select a subset as options
                options = random.sample([key for key in keyphrases if key != keyword], k=num_options - 1)
                
                # Add the keyword to the options
                options.append(keyword)
                
                # Shuffle the options
                random.shuffle(options)
                
                # Find the index of the correct answer
                correct_index = options.index(keyword)
                
                # Format the options list with option numbers and mark the correct answer
                options_str = "\n".join([f"{chr(65 + i)}. {option} {'(Correct Answer)' if i == correct_index else ''}" for i, option in enumerate(options)])
                
                mcq_with_answer = f"{question}\nOptions:\n{options_str}"
                mcqs_with_answers.append(mcq_with_answer)
    return mcqs_with_answers

# File path to the text containing information
file_path = "/Users/jasmithamareddy/Desktop/example.txt"

# Extract keyphrases from the file
keyphrases = get_keyphrases(file_path)

# Number of questions to generate
num_questions = 10

# Generate MCQs with answers
mcqs_with_answers = []
for _ in range(num_questions):
    mcqs_with_answers.extend(generate_mcqs_with_answers(keyword_sentence_mapping, keyphrases))

# Print MCQs with answers
for i, mcq_with_answer in enumerate(mcqs_with_answers, 1):
    print(f"MCQ {i}:\n{mcq_with_answer}\n")


MCQ 1:
The ____ is a crafted form in its own right.
Options:
A. plot 
B. story 
C. short story (Correct Answer)
D. point 

MCQ 2:
While the ____ is largely distinct from the novel or novella/short novel, authors generally draw from a common pool of literary techniques.
Options:
A. novel 
B. point 
C. short story (Correct Answer)
D. genre 

MCQ 3:
[citation needed] The ____ is sometimes referred to as a genre Determining what exactly defines a ____ remains problematic.
Options:
A. short story (Correct Answer)
B. plot 

MCQ 4:
A classic definition of a ____ is that one should be able to read it in one sitting, a point most notably made in Edgar Allan Poe's essay "The Philosophy of Composition" (1846) H.G.
Options:
A. short story (Correct Answer)
B. story 

MCQ 5:
Wells described the purpose of the ____ as "The jolly art, of making something very bright and moving; it may be horrible or pathetic or funny or profoundly illuminating, having only this essential, that it should take from fift